In [1]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
import datetime as datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "keyur",
            "password": "Keyur@17061996",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [3]:
disbursed_cases = """
    select * from (select 
    la.id,
    la.user_id,
    ou.full_name,
    ou.pin_code,
    o.phone_number,
    o.pan_number,
    ou.email,
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False
    )
    where disbursement_date between '2023-01-16' and '2023-01-31'
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

(1312, 8)

In [4]:
total_disbursed_cases.head()

,id,user_id,full_name,pin_code,phone_number,pan_number,email,disbursement_date
0,a87f1d24-76ca-483a-8cac-9bb855ec0a19,fb45b4d9-af5e-4339-b95b-57f65be66847,Muthumari S,600001,9176352553,GEHPM9441C,BHAVANIMARI2013@GMAIL.COM,2023-01-30
1,21842acc-ad27-498b-85bd-f695238fa7bf,ab4ec913-2431-4665-9d42-9ced8fee0c4f,Atiya Jabeen,500017,8328484445,AZJPA7250Q,MOHAMMEDMUMTAZULHAQUE1979@GMAIL.COM,2023-01-30
2,b0d4bb02-d1e1-42c1-bc28-2597eeb3bf09,7ad6ffd9-edeb-4aa2-800b-5a8cc2bee6e5,Naveen Sumersingh,127308,9992037532,BWTPN8157K,noemail@noemail.com,2023-01-30
3,6c186ce3-0f46-4b1b-9230-901e6bfc60fe,4cadb33b-bea8-41f6-9a9e-16796374432f,Deepak Kumar,243001,9870698471,HSXPK9021F,DEEPAK.DEEPAK.VERMA9433@GMAIL.COM,2023-01-24
4,cd88f593-aa97-42d1-84bd-2bd187f221fc,d01c8f22-240b-49b6-a556-af64ff2c4a87,Prem Chand Parjapati,829134,7488250028,ARKPP2469F,PREMPRAJAPATI557@GMAIL.COM,2023-01-20


In [5]:
total_disbursed_cases.to_csv('total_disbursed_cases.csv', index=False)

#### Rejected cases

In [8]:
ntc_cases = """
    select * from (select 
    la.id,
    la.user_id,
    ou.full_name,
    ou.pin_code,
    o.phone_number,
    o.pan_number,
    ou.email,
    rank() over(partition by od.user_id order by od.created_at desc) as rnk
    from analytics.offers.offer_details od
    where od.bureau_score = -1
    and od.created_at >= '2024-01-01'
    )
    where rnk = 1;
"""

total_ntc_cases = extractorFromQuery().queryExecutor(ntc_cases)
total_ntc_cases.shape

(3, 8)

In [9]:
total_ntc_cases

,id,user_id,full_name,pin_code,phone_number,pan_number,email,rnk
0,862c56fe-9712-4268-8e7e-de82614a734a,656e17e4-fa73-445d-9d87-63967cc1cee9,Sukhdeo Singh,305901,9660223191,GKIPS9356B,SDEVMSDRAWAT@GMAIL.COM,1
1,e770ca69-e27a-4186-bebb-bb1a97ce1e58,656e17e4-fa73-445d-9d87-63967cc1cee9,Sukhdev Singh,305924,9660223191,GKIPS9356B,noemail@noemail.com,1
2,b2e14432-71c7-43af-a992-34258b31d7c9,b062e354-b861-452c-b821-20d525dabcfd,Yashwant Singh,226001,9616380853,FYCPS3875N,SINGHYASHWANT445@GMAIL.COM,1


In [28]:
Q = """select rejection_reason, count(rejection_reason)
from niro_rds.niro_data.core__public__loan_application
where loan_application_status ='REJECTED'
group by rejection_reason
order by count(rejection_reason) desc
"""

tp = extractorFromQuery().queryExecutor(Q)
tp.shape

(24841, 2)

In [29]:
tp.to_csv('Rejected.csv', index=False)